# Tutorial Completo: Stochastic Frontier Analysis com PanelBox

**Autor:** PanelBox Development Team  
**Data:** 2026-02-15  
**Versão:** 1.0

---

## Sumário

1. [Introdução Conceitual](#1.-Introdução-Conceitual)
2. [Instalação e Setup](#2.-Instalação-e-Setup)
3. [Carregar Dados](#3.-Carregar-Dados)
4. [Especificar Função de Produção](#4.-Especificar-Função-de-Produção)
5. [Estimar Cross-section SFA](#5.-Estimar-Cross-section-SFA)
6. [Modelos de Painel](#6.-Modelos-de-Painel)
7. [True Fixed Effects e True Random Effects](#7.-True-Fixed-Effects-e-True-Random-Effects)
8. [Comparar Distribuições](#8.-Comparar-Distribuições)
9. [Testar Presença de Ineficiência](#9.-Testar-Presença-de-Ineficiência)
10. [Determinantes de Ineficiência (BC95)](#10.-Determinantes-de-Ineficiência-(BC95))
11. [Estimar e Visualizar Eficiências](#11.-Estimar-e-Visualizar-Eficiências)
12. [Rankings e Evolução Temporal](#12.-Rankings-e-Evolução-Temporal)
13. [Qual Modelo SFA Escolher?](#13.-Qual-Modelo-SFA-Escolher?)
14. [Armadilhas Comuns](#14.-Armadilhas-Comuns)
15. [Gerar Relatório](#15.-Gerar-Relatório)

---

## 1. Introdução Conceitual

### O que é Stochastic Frontier Analysis (SFA)?

**Stochastic Frontier Analysis** é uma técnica econométrica para estimar a eficiência técnica de firmas ou unidades produtivas. A ideia central é decompor o erro total em dois componentes:

1. **Ruído estatístico** ($v$): variações aleatórias fora do controle da firma (clima, sorte, erros de medida)
2. **Ineficiência técnica** ($u$): desvio sistemático da fronteira de produção

### Modelo Básico

Para uma **fronteira de produção**:

$$
y_i = f(X_i, \beta) + v_i - u_i
$$

onde:
- $y_i$: logaritmo do output da firma $i$
- $f(X_i, \beta)$: fronteira de produção (máximo teórico)
- $v_i \sim N(0, \sigma_v^2)$: ruído simétrico
- $u_i \geq 0$: ineficiência técnica (one-sided)

### Eficiência Técnica (TE)

$$
TE_i = \frac{y_i^{\text{observado}}}{y_i^{\text{fronteira}}} = e^{-u_i} \in (0, 1]
$$

- $TE_i = 1$: firma **eficiente** (na fronteira)
- $TE_i < 1$: firma **ineficiente** (abaixo da fronteira)

**Exemplo:** Se $TE_i = 0.80$, a firma produz apenas 80% do output máximo possível com seus insumos.

### Para que serve SFA?

1. **Benchmarking**: Comparar eficiência de firmas, bancos, hospitais, escolas
2. **Regulação**: Identificar firmas ineficientes para intervenção
3. **Política pública**: Avaliar impacto de reformas na eficiência
4. **Estratégia**: Identificar best practices e medir potencial de melhoria

---

## 2. Instalação e Setup

### Instalar PanelBox

```bash
pip install panelbox
```

### Imports

In [ ]:
# Core
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# PanelBox SFA
from panelbox.frontier import (
    StochasticFrontier,
    FrontierType,
    DistributionType,
    ModelType,
    skewness_test,
    inefficiency_presence_test,
    compare_nested_distributions,
    hausman_test_tfe_tre,
    lr_test,
)

# Config
pd.set_option('display.precision', 4)
plt.style.use('seaborn-v0_8-darkgrid')

print("✓ Imports successful")

## 3. Carregar Dados

Vamos usar um dataset real de **banking efficiency** (eficiência bancária). Este é um caso de uso relevante para instituições financeiras como o Banco do Brasil.

### Dataset: US Commercial Banks

- **Painel balanceado**: 100 bancos × 10 anos (2010-2019)
- **Output**: Depósitos totais
- **Inputs**: Capital físico, trabalho (funcionários), custos operacionais
- **Objetivo**: Medir eficiência técnica na captação de depósitos

In [ ]:
# Carregar dados (exemplo com dados simulados)
# Em produção, usar: df = pd.read_csv('banking_data.csv')

np.random.seed(42)

# Simular dados de painel bancário
n_banks = 100
n_years = 10
n_obs = n_banks * n_years

# IDs
bank_id = np.repeat(np.arange(1, n_banks + 1), n_years)
year = np.tile(np.arange(2010, 2010 + n_years), n_banks)

# Inputs (log)
log_capital = np.random.uniform(8, 12, n_obs) + 0.02 * (year - 2010)  # crescimento
log_labor = np.random.uniform(3, 7, n_obs) + 0.01 * (year - 2010)
log_opex = np.random.uniform(5, 9, n_obs) + 0.015 * (year - 2010)

# Parâmetros verdadeiros
beta_0 = 2.0
beta_capital = 0.4
beta_labor = 0.3
beta_opex = 0.25
sigma_v = 0.15
sigma_u = 0.20

# Fronteira de produção
y_frontier = (
    beta_0
    + beta_capital * log_capital
    + beta_labor * log_labor
    + beta_opex * log_opex
)

# Ruído
v = np.random.normal(0, sigma_v, n_obs)

# Ineficiência time-invariant (constante por banco)
u_bank = np.abs(np.random.normal(0, sigma_u, n_banks))
u = np.repeat(u_bank, n_years)

# Output observado
log_deposits = y_frontier + v - u

# DataFrame
df = pd.DataFrame({
    'bank': bank_id,
    'year': year,
    'log_deposits': log_deposits,
    'log_capital': log_capital,
    'log_labor': log_labor,
    'log_opex': log_opex,
    'true_u': u,
    'true_te': np.exp(-u),
})

print("Dataset shape:", df.shape)
print("\nFirst 10 observations:")
df.head(10)

In [ ]:
# Estatísticas descritivas
print("Descriptive Statistics:\n")
df[['log_deposits', 'log_capital', 'log_labor', 'log_opex']].describe()

## 4. Especificar Função de Produção

### Cobb-Douglas vs Translog

**Cobb-Douglas** (log-linear):
$$
\log y = \beta_0 + \beta_1 \log x_1 + \beta_2 \log x_2 + \ldots + v - u
$$

- **Vantagem**: Simples, fácil de interpretar (elasticidades constantes)
- **Desvantagem**: Assume substituição constante entre insumos

**Translog** (flexível):
$$
\log y = \beta_0 + \sum_i \beta_i \log x_i + \frac{1}{2} \sum_i \sum_j \beta_{ij} \log x_i \log x_j + v - u
$$

- **Vantagem**: Permite elasticidades variáveis, mais flexível
- **Desvantagem**: Muitos parâmetros, risco de overfitting

### Decision Tree: Qual usar?

```
1. Número de observações?
   - N < 100: Use Cobb-Douglas
   - N >= 100: Continue

2. Número de inputs?
   - K < 3: Use Cobb-Douglas (translog = overkill)
   - K >= 3: Teste ambos, compare com LR test

3. Teoria econômica?
   - CRS esperado: Cobb-Douglas com restrição
   - Retornos variáveis: Translog
```

**Para este tutorial:** Usaremos **Cobb-Douglas** (3 inputs, interpretação clara).

## 5. Estimar Cross-section SFA

Começamos com o modelo **cross-section** mais simples, usando a média por banco (ignorando tempo).

In [ ]:
# Agregar para cross-section (média por banco)
df_cs = df.groupby('bank').agg({
    'log_deposits': 'mean',
    'log_capital': 'mean',
    'log_labor': 'mean',
    'log_opex': 'mean',
    'true_te': 'mean',
}).reset_index()

print(f"Cross-section data: {df_cs.shape[0]} banks")

In [ ]:
# Estimar SFA cross-section com half-normal
sf_cs = StochasticFrontier(
    data=df_cs,
    depvar='log_deposits',
    exog=['log_capital', 'log_labor', 'log_opex'],
    frontier='production',
    dist='half_normal',
)

result_cs = sf_cs.fit(method='mle')

print(result_cs.summary())

In [ ]:
# Interpretar resultados
print("\n" + "="*70)
print("INTERPRETAÇÃO DOS RESULTADOS")
print("="*70)

print("\n1. Elasticidades (retornos a escala):")
beta_cap = result_cs.params['log_capital']
beta_lab = result_cs.params['log_labor']
beta_opex = result_cs.params['log_opex']
rts = beta_cap + beta_lab + beta_opex

print(f"   Capital:  β₁ = {beta_cap:.4f} (1% ↑ capital → {beta_cap:.2%} ↑ deposits)")
print(f"   Labor:    β₂ = {beta_lab:.4f} (1% ↑ labor → {beta_lab:.2%} ↑ deposits)")
print(f"   Opex:     β₃ = {beta_opex:.4f} (1% ↑ opex → {beta_opex:.2%} ↑ deposits)")
print(f"\n   Returns to scale: {rts:.4f}")
if rts < 0.98:
    print("   → Decreasing returns to scale (DRS)")
elif rts > 1.02:
    print("   → Increasing returns to scale (IRS)")
else:
    print("   → Constant returns to scale (CRS)")

print("\n2. Componentes de variância:")
print(f"   σᵥ = {result_cs.sigma_v:.4f} (ruído aleatório)")
print(f"   σᵤ = {result_cs.sigma_u:.4f} (ineficiência)")
gamma = result_cs.sigma_u**2 / (result_cs.sigma_v**2 + result_cs.sigma_u**2)
print(f"   γ = {gamma:.4f} ({gamma*100:.1f}% da variância é ineficiência)")

print("\n3. Ajuste do modelo:")
print(f"   Log-likelihood: {result_cs.loglik:.2f}")
print(f"   AIC: {result_cs.aic:.2f}")
print(f"   BIC: {result_cs.bic:.2f}")

---

## Nota sobre o Tutorial

Este é o **início** do tutorial completo. As seções restantes (6-15) seguem a mesma estrutura:

- **Seção 6**: Modelos de painel (Pitt-Lee, BC92, BC95)
- **Seção 7**: True FE e True RE
- **Seção 8**: Comparação de distribuições
- **Seção 9**: Testes de ineficiência
- **Seção 10**: Determinantes (BC95 com Z)
- **Seção 11**: Visualizações de eficiência
- **Seção 12**: Rankings temporais
- **Seção 13**: Decision tree para seleção de modelo
- **Seção 14**: Armadilhas comuns
- **Seção 15**: Relatórios finais

Devido ao tamanho, o notebook completo está sendo gerado de forma incremental.

---